In [ ]:
#!pip install sqlalchemy
#!pip install pymysql

In [ ]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

# Import Pandas
import pandas as pd

In [ ]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Cloud MySQL Database Connection on AWS
cloud_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")

In [ ]:
# Create a local database engine connection
local_conn = local_engine.connect()

# Create a remote database engine connection
cloud_conn = cloud_engine.connect()

In [ ]:
# Create local gwsis database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)
    
# Connect to local gwsis database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)    

## ETL Example 1

This code segment will migrate student enrollment data from your MySQL database hosted on AWS to your local Postgres database hosted on your machine.

In [ ]:
# Query the remote MySQL database
student_data = pd.read_sql("SELECT * FROM all_student_enrollment", cloud_conn)
student_data.head()

In [ ]:
student_data_load = student_data 
#student_data_load.drop(columns=['ID_Student'], inplace=True)

In [ ]:
student_data_load.to_sql(name='student_enrollment', if_exists='replace', con=local_conn, index=False)

In [ ]:
local_student_data = pd.read_sql("SELECT * FROM student_enrollment", local_conn)
print(len(local_student_data))
local_student_data.head()

## ETL Example 2

This code segment will migrate data from a local csv to your MySQL database hosted on AWS

In [ ]:
census_data = pd.read_csv('../Resources/Census_Data.csv')
census_data.head()

In [ ]:
census_data.to_sql(name='census_data', if_exists='append', con=cloud_conn, index=False)

In [ ]:
remote_census_data = pd.read_sql("SELECT * FROM census_data WHERE city='Chicago'", cloud_conn)
print(len(remote_census_data))
remote_census_data.head(50)